In [6]:
import os
import sys
import pandas as pd
import networkx as nx
import pickle

In [7]:
DG = nx.DiGraph()

In [15]:
with open('pgres.pickle', 'rb') as handle:
    pgres = pickle.load(handle)

invpg = {pgres[k]:k for k in pgres.keys()}

In [67]:
teams = ["Memphis", "Boise St."]
teams = ["Alabama", "Notre Dame"]
teams = ["North Carolina", "Marquette"]
teams = ["Seton Hall", "TCU"]
teams = ["Colorado St.", "Michigan"]
teams = ["Duke", "Boise St."]
teams = ["San Diego St.", "Creighton"]
teams = ["Seton Hall", "TCU"]
teams = ["Memphis", "Boise St."]
teams = ["Arkansas", "UConn"]
teams = ["Alabama", "Texas Tech"]
teams = ["UCLA", "Saint Mary's (CA)"]
teams = ["Houston", "Illinois"]
teams = ["Colorado St.", "Tennessee"]
teams = ["Providence", "Iowa"]
teams = ["LSU", "Wisconsin"]
teams = ["Auburn", "USC"]
teams = ["Gonzaga", "UConn"]
teams = ["Texas Tech", "Duke"]
teams = ["UCLA", "Baylor"]
teams = ["Kentucky", "Purdue"]
teams = ["Arizona", "Houston"]
teams = ["Villanova", "Tennessee"]
teams = ["Kansas", "Iowa"]
teams = ["Auburn", "LSU"]
teams = ["Gonzaga", "Duke"]
teams = ["Kentucky", "UCLA"]
teams = ["Houston", "Tennessee"]
teams = ["Kansas", "Auburn"]
teams = ["Gonzaga", "Kentucky"]
teams = ["Houston", "Auburn"]
teams = ["Gonzaga", "Auburn"]
for k in teams:
    print(f" {k} {pgres[k]*1000}")
    

 Gonzaga 0.5017987767973922
 Auburn 0.49099290902270376


In [46]:
for i, k in enumerate(sorted(invpg.keys(), reverse=False)[:100]):
    print(f"{i}:  {invpg[k]} {k*1000}")

0:  Auburn 0.49099290902270376
1:  Houston 0.4924992677655311
2:  San Francisco 0.494899305122631
3:  Gonzaga 0.5017987767973922
4:  Kentucky 0.5161893027883739
5:  UCLA 0.5220917283613
6:  Kansas 0.5254268293900278
7:  UConn 0.5392321879227823
8:  Arizona 0.5443379657296323
9:  Southern California 0.5482711988242109
10:  Duke 0.5488451711400882
11:  Baylor 0.5508928539102989
12:  North Texas 0.5554458614742527
13:  Texas Tech 0.564535487332545
14:  Tennessee 0.565312372434988
15:  Seton Hall 0.5745641217526316
16:  Purdue 0.5749603867050523
17:  Villanova 0.5768791836631262
18:  Saint Mary's (CA) 0.5812074980115298
19:  Texas 0.5904162263570656
20:  Colorado St. 0.5934259417727274
21:  Iowa 0.5959505985754693
22:  LSU 0.5991911041166486
23:  Davidson 0.6026840973476743
24:  Loyola Chicago 0.606105829668153
25:  San Diego St. 0.6228025398986007
26:  Wisconsin 0.6240986258490314
27:  Murray St. 0.6247088922864691
28:  Kansas St. 0.6334281491290987
29:  TCU 0.6407335989424247
30:  Oklaho

In [12]:
teams = ["Duke", "Gonzaga", "Kansas", "Arizona"]

for team in teams:
    print(f" {team} {pgres[team]*1000}")




 Duke 0.5488451711400882
 Gonzaga 0.5017987767973922
 Kansas 0.5254268293900278
 Arizona 0.5443379657296323


In [47]:
sdf = pd.read_csv("mmscores.csv")
sdf.head(2)
sdf.dtypes

Team1      object
Team2      object
Score1    float64
Score2    float64
date       object
dtype: object

In [48]:

scols = ["Score1", "Score2"]
df = sdf.copy()
df["winnerScore"] = 0
df["loserScore"] = 0
df["winnerTeam"] = None
df["loserTeam"] = None
df["tie"] = False
for idx, ser in df.iterrows():
    if ser["Score1"] > ser["Score2"]:
        df.at[idx, "winnerTeam"] = ser["Team1"]
        df.at[idx, "loserTeam"] = ser["Team2"] 
        df.at[idx, "winnerScore"] = ser["Score1"]
        df.at[idx, "loserScore"] = ser["Score2"]         
    elif ser["Score1"] < ser["Score2"] :
        df.at[idx, "winnerTeam"] = ser["Team2"]
        df.at[idx, "loserTeam"] = ser["Team1"] 
        df.at[idx, "winnerScore"] = ser["Score2"]
        df.at[idx, "loserScore"] = ser["Score1"] 
    else:
        df.at[idx, "tie"] = True
df.head()

,Team1,Team2,Score1,Score2,date,winnerScore,loserScore,winnerTeam,loserTeam,tie
0,Bryant,Wright St.,59.0,67.0,2022-03-16,67,59,Wright St.,Bryant,False
1,Dayton,Toledo,57.0,40.0,2022-03-16,57,40,Dayton,Toledo,False
2,Virginia,Mississippi St.,32.0,32.0,2022-03-16,0,0,None,None,True
3,UMES,Coastal Carolina,27.0,40.0,2022-03-16,40,27,Coastal Carolina,UMES,False
4,Morgan St.,Youngstown St.,6.0,13.0,2022-03-16,13,6,Youngstown St.,Morgan St.,False


In [49]:
all_teams = list(set(df.Team1.unique()).union(df.Team2.unique()))
len(all_teams)

796

In [51]:
wins = pd.DataFrame(df.winnerTeam.value_counts())
losses = pd.DataFrame(df.loserTeam.value_counts())
ties1 = pd.DataFrame(df.loc[df["tie"]].Team1.value_counts())
ties2 = pd.DataFrame(df.loc[df["tie"]].Team2.value_counts())

ties = ties1.join(ties2)
ties["cnt"] = ties.sum(axis=1)
ties.drop(columns=["Team1", "Team2"], inplace=True)

wldf = wins.join(losses)
wldf = wldf.join(ties)
wldf.reset_index(inplace=True)

wldf.columns = ["team", "wins", "losses", "ties",]
wldf.fillna(0, inplace=True)
wldf["games"] = wldf[["wins", "losses", "ties"]].sum(axis=1)
wldf.sort_values(by="wins", inplace=True)
wldf.tail(45)

,team,wins,losses,ties,games
54,San Diego St.,24,8.0,1.0,33.0
49,Illinois,24,9.0,0.0,33.0
26,Texas A&M,25,12.0,0.0,37.0
27,Drake,25,9.0,0.0,34.0
28,Iona,25,7.0,1.0,33.0
29,Belmont,25,8.0,0.0,33.0
30,Saint Louis,25,11.0,0.0,36.0
31,Arkansas,25,8.0,0.0,33.0
32,Louisiana Tech,25,11.0,0.0,36.0
33,Loyola Chicago,25,7.0,0.0,32.0


In [75]:
realdf = wldf.loc[wldf["games"] > 20].copy()
realdf.sort_values(by="wins", inplace=True, ascending=False)
display(realdf.head(2))
realteams = realdf["team"].values
print(realdf.shape)

,team,wins,losses,ties,games
0,Arizona,32,3.0,0.0,35.0
1,Houston,31,5.0,0.0,36.0


(358, 5)


In [82]:
sqdf = pd.DataFrame(columns=teams, index=teams)
for idx, ser in df.iterrows():
    if ser["winnerTeam"] in realteams and ser["loserTeam"] in realteams:
        sqdf.at[ser["winnerTeam"], ser["loserTeam"]] = ser["winnerScore"] - ser["loserScore"]
        sqdf.at[ser["loserTeam"], ser["winnerTeam"]] = ser["loserScore"] - ser["winnerScore"]    

In [84]:
sqdf.head()
#display(sqdf.sum(axis=1))
#display(sqdf.sum(axis=0))

,Arizona,Houston,South Dakota St.,Murray St.,Vermont,Kansas,UAB,Toledo,Auburn,Kentucky,...,Fairleigh Dickinson,Eastern Ill.,William & Mary,Green Bay,IUPUI,Columbia,Oregon St.,Delaware St.,Lamar University,Mississippi Val.
Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25,NaN,NaN,NaN
Houston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
South Dakota St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Murray St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-13,NaN,...,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df.shape

(6102, 10)

In [89]:
sqdf.shape[0] * sqdf.shape[1] - sqdf.isna().sum().sum()

7396